# MPNet Evaluation on VoiceBank+DEMAND (PESQ)

In [1]:
import os
import json
import torch
import librosa
import numpy as np
from tqdm.auto import tqdm
from models.mpnet.discriminator import cal_pesq

In [2]:
from utils import AttrDict
from models.mpnet.model import MPNet
from utils.dataset import mag_pha_stft, mag_pha_istft

In [3]:
CHECKPOINT_PATH = "experiments/best_pesq.pt"
CONFIG_PATH = "models/mpnet/config.json"

TESTSET_CLEAN_DIR = "VoiceBank+DEMAND/wavs_clean"
TESTSET_NOISY_DIR = "VoiceBank+DEMAND/wavs_noisy"
TEST_LIST_FILE = "VoiceBank+DEMAND/test.txt"

# Override segment_size (16kHz: 0.5s=8000, 0.75s=12000, 1.0s=16000)
SEGMENT_SIZE = 16000

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Segment size: {SEGMENT_SIZE} samples ({SEGMENT_SIZE/16000:.2f}s)")

Using device: mps
Segment size: 16000 samples (1.00s)


In [4]:
with open(CONFIG_PATH) as f:
    h = AttrDict(json.load(f))

h.segment_size = SEGMENT_SIZE

model = MPNet(h, num_tsblocks=h.num_tsconformers).to(device)
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=True)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print(f"Model loaded: {sum(p.numel() for p in model.parameters()):,} params")
print(f"Checkpoint epoch: {checkpoint.get('epoch', 'N/A')}, best_pesq: {checkpoint.get('best_pesq', 'N/A'):.3f}")

Model loaded: 2,263,372 params
Checkpoint epoch: 27, best_pesq: 3.620


In [5]:
with open(TEST_LIST_FILE) as f:
    test_files = [line.strip().split("|")[0] for line in f if line.strip()]
print(f"Test files: {len(test_files)}")

Test files: 824


In [6]:
def denoise(noisy_wav):
    noisy = torch.FloatTensor(noisy_wav).to(device)
    norm = torch.sqrt(len(noisy) / torch.sum(noisy ** 2.0))
    noisy = (noisy * norm).unsqueeze(0)
    
    noisy_amp, noisy_pha, _ = mag_pha_stft(noisy, h.n_fft, h.hop_size, h.win_size, h.compress_factor)
    
    with torch.no_grad():
        amp_g, pha_g, _ = model(noisy_amp, noisy_pha)
    
    audio_g = mag_pha_istft(amp_g, pha_g, h.n_fft, h.hop_size, h.win_size, h.compress_factor)
    return (audio_g / norm).squeeze().cpu().numpy()

In [ ]:
pesq_enhanced, pesq_noisy = [], []

for filename in tqdm(test_files):
    clean_path = os.path.join(TESTSET_CLEAN_DIR, filename + ".wav")
    noisy_path = os.path.join(TESTSET_NOISY_DIR, filename + ".wav")
    
    clean_wav, _ = librosa.load(clean_path, sr=h.sampling_rate)
    noisy_wav, _ = librosa.load(noisy_path, sr=h.sampling_rate)
    
    denoised_wav = denoise(noisy_wav)
    
    pesq_enhanced.append(cal_pesq(clean_wav, denoised_wav, h.sampling_rate))
    # pesq_noisy.append(cal_pesq(clean_wav, noisy_wav, h.sampling_rate))

  0%|          | 0/824 [00:00<?, ?it/s]

In [8]:
print(f"Evaluated: {len(pesq_enhanced)} files")
print(f"\nPESQ Noisy: {np.mean(pesq_noisy):.4f}")
print(f"PESQ Enhanced: {np.mean(pesq_enhanced):.4f}")

Evaluated: 824 files

PESQ Noisy: 1.9707
PESQ Enhanced: 3.3902
